In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

customers_path = os.path.join(os.getcwd(), 'Customers.csv')
products_path = os.path.join(os.getcwd(), 'Products.csv')
transactions_path = os.path.join(os.getcwd(), 'Transactions.csv')

customers = pd.read_csv(customers_path, dtype={'CustomerID': 'string', 'Region': 'string', 'CustomerName': 'string'})
products = pd.read_csv(products_path, dtype={'ProductID': 'string', 'Category': 'string', 'ProductName': 'string', 'Price': 'float'})
transactions = pd.read_csv(transactions_path, dtype={'TransactionID': 'string', 'CustomerID': 'string', 'ProductID': 'string', 'Quantity': 'int', 'TotalValue': 'float'}, parse_dates=['TransactionDate'])

customers.dropna(inplace=True)
products.dropna(inplace=True)
transactions.dropna(inplace=True)

customers.drop_duplicates(inplace=True)
products.drop_duplicates(inplace=True)
transactions.drop_duplicates(inplace=True)

merged_data = transactions.merge(customers, on="CustomerID", how="inner").merge(products, on="ProductID", how="inner")

merged_data['TransactionDate'] = pd.to_datetime(merged_data['TransactionDate'], errors='coerce')
merged_data.dropna(subset=['TransactionDate'], inplace=True)
merged_data['MonthYear'] = merged_data['TransactionDate'].dt.to_period('M')

plt.figure(figsize=(10, 6))
sns.histplot(merged_data['TotalValue'].dropna().sample(frac=0.1), bins=30, kde=True)
plt.title("Distribution of Transaction TotalValue")
plt.xlabel("TotalValue")
plt.ylabel("Frequency")
plt.show()

top_products = merged_data.groupby('ProductName')['Quantity'].sum().sort_values(ascending=False).head(10)
plt.figure(figsize=(12, 6))
top_products.plot(kind='bar', color='skyblue')
plt.title("Top 10 Most Sold Products")
plt.xlabel("Product Name")
plt.ylabel("Total Quantity Sold")
plt.show()

region_sales = merged_data.groupby('Region')['TotalValue'].sum().sort_values(ascending=False)
plt.figure(figsize=(10, 6))
region_sales.plot(kind='bar', color='coral')
plt.title("Total Sales by Region")
plt.xlabel("Region")
plt.ylabel("Total Sales Value")
plt.show()

monthly_sales = merged_data.groupby('MonthYear')['TotalValue'].sum()
plt.figure(figsize=(14, 7))
monthly_sales.plot(marker='o', linestyle='-', color='green')
plt.title("Monthly Sales Trends")
plt.xlabel("Month-Year")
plt.ylabel("Total Sales Value")
plt.grid()
plt.show()

category_sales = merged_data.groupby('Category')['TotalValue'].sum().sort_values(ascending=False)
plt.figure(figsize=(12, 6))
category_sales.plot(kind='bar', color='purple')
plt.title("Category-Wise Total Sales")
plt.xlabel("Category")
plt.ylabel("Total Sales Value")
plt.show()